En el nodo "Maestro" hacer un programa en Python (en consola, modo texto) con bibliotecas MPI que se llamará "lector.py" y que haga lo siguiente:
+ Enviar un mensaje al nodo "Esclavo" para que empiece su procesamiento.
+ Leer los primeros 3,000,000 de registros de nuestro dataset "210516COVID19MEXICO.csv".
+ Contar los registros de los pacientes que hayan fallecido.
+ Calcular el promedio de edad de los pacientes fallecidos.
+ Esperar por los resultados que recibirá del nodo "Esclavo".
+ Imprimir los datos que recibe nodo "Esclavo".
+ Calcular e imprimir el total de fallecido por COVID-19 (sumando los datos de ambos nodos).
+ Calcular e imprimir el promedio total de pacientes fallecidos por COVID-19 (promediando los datos de ambos nodos).


In [1]:
import pandas as pd

In [2]:
file = pd.read_csv('210516COVID19MEXICO.csv')

In [3]:
file.shape
registros = len(file)
print(f"La cantidad de registros es {registros}")

La cantidad de registros es 6852082


In [4]:
file.head(5)

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2021-05-16,z482b8,1,12,9,2,9,9,12,1,...,2,2,97,2,97,1,99,México,97,97
1,2021-05-16,z49a69,1,12,23,1,23,23,4,2,...,1,2,97,2,97,2,99,México,97,1
2,2021-05-16,z23d9d,1,12,22,2,24,22,9,1,...,2,2,97,2,97,6,99,México,97,97
3,2021-05-16,z24953,1,12,9,1,9,9,10,1,...,1,1,2,2,97,7,99,México,97,97
4,2021-05-16,zz8e77,1,12,9,2,9,9,2,1,...,2,2,97,2,97,6,99,México,97,97


In [5]:
mitad = int(registros / 2)
print(f'Cada dataser nuevo tendrá {mitad} registros')

Cada dataser nuevo tendrá 3426041 registros


In [6]:
file1 = file.head(mitad)
file2 = file.tail(mitad)

print('Registros del archivo1 = ',len(file1))
print('Registros del archivo2 = ',len(file2))

Registros del archivo1 =  3426041
Registros del archivo2 =  3426041


In [7]:
file1.head(5)


,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2021-05-16,z482b8,1,12,9,2,9,9,12,1,...,2,2,97,2,97,1,99,México,97,97
1,2021-05-16,z49a69,1,12,23,1,23,23,4,2,...,1,2,97,2,97,2,99,México,97,1
2,2021-05-16,z23d9d,1,12,22,2,24,22,9,1,...,2,2,97,2,97,6,99,México,97,97
3,2021-05-16,z24953,1,12,9,1,9,9,10,1,...,1,1,2,2,97,7,99,México,97,97
4,2021-05-16,zz8e77,1,12,9,2,9,9,2,1,...,2,2,97,2,97,6,99,México,97,97


In [8]:
file2.head(5)
file2.columns


Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [9]:
new_file1 = file1[['EDAD','FECHA_DEF']]
new_file2 = file2[['EDAD','FECHA_DEF']]

In [10]:
new_file1.head(5)

,EDAD,FECHA_DEF
0,41,9999-99-99
1,66,2020-07-21
2,29,9999-99-99
3,40,9999-99-99
4,34,9999-99-99


In [11]:
maestro = new_file1[new_file1['FECHA_DEF'] != '9999-99-99']

In [12]:
esclavo = new_file2[new_file2['FECHA_DEF'] != '9999-99-99']

In [13]:
maestro.shape


(180429, 2)

In [14]:
#esclavo.shape
muertes1 = len(maestro)
muertes2 = len(esclavo)

In [15]:
print(f'Muertes contadas en el programa 1: {muertes1}')
print(f'Muertes contadas en el programa 2: {muertes2}')
print(f'TOTAL de muertes: {muertes1+muertes2}')

Muertes contadas en el programa 1: 180429
Muertes contadas en el programa 2: 109471
TOTAL de muertes: 289900


In [16]:
promedio1 = int(maestro['EDAD'].mean())
promedio2 = int(esclavo['EDAD'].mean())

print(f"Promedio 1 = {promedio1}")
print(f"Promedio 2 = {promedio2}")
promedio_total = int((promedio1+promedio2 )/ 2)

print(f"Promedio TOTAL = {promedio_total}")

Promedio 1 = 62
Promedio 2 = 64
Promedio TOTAL = 63


In [17]:
"{0:.2f}".format(maestro['EDAD'].mean())

'62.53'